---
### Walk Forward: A Realistic Approach to Backtesting
---
#### I. Load the data

In [64]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', parse_dates=['Date'], index_col=0)
df = df.drop(columns='change_tomorrow_direction')
df.head(n=5)

,Close,High,Low,Open,Volume,change_tomorrow
Date,,,,,,
2016-12-08,55.181126,55.696671,55.027369,55.443420,21220800,1.549151
2016-12-09,56.049416,56.067505,55.289669,55.334891,27349400,0.321666
2016-12-12,56.230289,56.347870,55.823285,55.913730,20198100,1.286169
2016-12-13,56.962929,57.360890,56.293630,56.528788,35718900,-0.478644
2016-12-14,56.691578,57.388013,56.555907,56.981005,30352700,-0.159789


<p align="center">
  <img src="screen/AWF-UWF.png" width="800"/>
</p>

**Walk Forward Validation (Time Series Cross-Validation)**

Walk Forward Validation is a strategy used to evaluate predictive models on time series data, where the order of data points matters.

**How it *works*:** Instead of randomly splitting the dataset, which would break the time dependency, *Walk Forward Validation* trains the model on a block of past data and tests it on a future period. The process is repeated by shifting the time window forward. This simulates a real-life forecasting scenario where only past data is available to predict future outcomes.

Example with 1000 days of data and 200 days reserved for testing in each split:

<div align="center">

<table>
  <thead>
    <tr>
      <th>Fold</th>
      <th>Training Days</th>
      <th>Testing Days</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>Day 1 to 800</td>
      <td>Day 801 to 1000</td>
    </tr>
    <tr>
      <td>2</td>
      <td>Day 1 to 801</td>
      <td>Day 802 to 1001</td>
    </tr>
    <tr>
      <td>3</td>
      <td>Day 1 to 802</td>
      <td>Day 803 to 1002</td>
    </tr>
    <tr>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
  </tbody>
</table>

</div>

**Anchored Walk-Forward Validation**

In this method, the training set always starts from the beginning of the dataset and grows with each split, while the test set moves forward in time. This approach reflects the idea of constantly incorporating all available historical data up to the prediction point. The training window is anchored to the start.

**Unanchored Walk-Forward Validation**

In this method, both the training and test windows move forward. The training set is no longer anchored to the beginning; instead, it may represent a fixed-size rolling window of recent observations. This approach is useful when older data becomes less relevant and the model should be trained only on the most recent information.

---
#### II. Walk Forward Validation

How `TimeSeriesSplit` works?

In [65]:
# Import TimeSeriesSplit for walk-forward cross-validation
from sklearn.model_selection import TimeSeriesSplit

In [66]:
# Create a TimeSeriesSplit object with a test set size of 200
# This means each split will reserve the last 200 time steps for testing
ts = TimeSeriesSplit(test_size=200)

# Generate the split indices (as an iterator of train/test indices)
splits = ts.split(X=df)

# Retrieve the first and second train/test splits from the iterator

# Each call to next(splits) gives you a tuple of two arrays:
# - The first array contains the indices for the training set
# - The second array contains the indices for the test set
# This maintains temporal order — training data always precedes test data.

split1 = next(splits)  # First fold: train on earliest data, test on the first 200 future points
split2 = next(splits)  # Second fold: train set extends further, test set moves forward

In [67]:
list_df_train = []
list_df_test = []

# Iterate through the time series splits
# ts.split(df) yields pairs of (train indices, test indices) for each fold
for index_train, index_test in ts.split(df):
    
    # Append the training set for the current split using index-based selection
    list_df_train.append(df.iloc[index_train])
    
    # Append the test set for the current split
    list_df_test.append(df.iloc[index_test])

---
#### III. Machine Learning Model

Separate the data:

1. Target: which variable do you want to predict?
2. Explanatory: which variables will you use to calculate the prediction?

In [68]:
y = df.change_tomorrow
X = df[['Open','High','Low','Close','Volume']]

In [69]:
list_df_train = []
list_df_test = []

for index_train, index_test in ts.split(df):
    X_train, y_train = X.iloc[index_train], y.iloc[index_train]
    X_test, y_test = X.iloc[index_test], y.iloc[index_test]

Simulate one computation of the ML model:

- Compute the model
- Calculate predictions on the test set
- Evaluate how good the model is

In [70]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

model_dt = DecisionTreeRegressor(max_depth=15, random_state=42)
model_dt.fit(X_train, y_train)

y_pred = model_dt.predict(X_test)
error_mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error on the test set: {error_mse:.4f}.")

Mean Squared Error on the test set: 2.3774.


Add the procedure inside the for loop.

In [71]:
model_dt = DecisionTreeRegressor(max_depth=15, random_state=42)

error_mse_list = []

for index_train, index_test in ts.split(df):
    X_train, y_train = X.iloc[index_train], y.iloc[index_train]
    X_test, y_test = X.iloc[index_test], y.iloc[index_test]
    
    model_dt.fit(X_train, y_train)
    
    y_pred = model_dt.predict(X_test)
    error_mse = mean_squared_error(y_test, y_pred)
    
    error_mse_list.append(error_mse)

for i, mse in enumerate(error_mse_list, start=1):
    print(f"Split {i}: MSE = {mse:.4f}.")

Split 1: MSE = 3.5955.
Split 2: MSE = 8.4399.
Split 3: MSE = 5.7309.
Split 4: MSE = 7.0429.
Split 5: MSE = 2.3774.


In [72]:
import  numpy as np

mean_mse = np.mean(error_mse_list)
print(f"Average Mean Squared Error across all splits: {mean_mse:.4f}.")

Average Mean Squared Error across all splits: 5.4373.


---
#### IV. Anchored Walk Forward evaluation in backtesting

Create a new strategy.

In [73]:
from backtesting import Strategy

In [74]:
class Regression(Strategy):
    limit_buy = 1
    limit_sell = -5
    
    n_train = 600
    coef_retrain = 200
    
    def init(self):
        self.model = DecisionTreeRegressor(max_depth=15, random_state=42)
        self.already_bought = False
        
        X_train = self.data.df.iloc[:self.n_train, :-1]
        y_train = self.data.df.iloc[:self.n_train, -1]
        
        self.model.fit(X=X_train, y=y_train)

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :-1]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [75]:
class WalkForwardAnchored(Regression):
    def next(self):
        
        # we don't take any action and move on to the following day
        if len(self.data) < self.n_train:
            return
        
        # we retrain the model each 200 days
        if len(self.data) % self.coef_retrain == 0:
            X_train = self.data.df.iloc[:, :-1]
            y_train = self.data.df.iloc[:, -1]

            self.model.fit(X_train, y_train)

            super().next()
            
        else:
            
            super().next()

Run the backtest.

In [76]:
from backtesting import Backtest

In [77]:
bt = Backtest(df, WalkForwardAnchored, cash=10000, commission=.002, exclusive_orders=True)

In [78]:
import multiprocessing as mp
mp.set_start_method('fork')

RuntimeError: context has already been set

In [79]:
# Run a parameter optimization on the backtest object using 'skopt' (Bayesian optimization)
stats_skopt, heatmap, optimize_result = bt.optimize(
    limit_buy=range(0, 6),         # Range of values to try for the buy limit threshold
    limit_sell=range(-6, 0),       # Range of values to try for the sell limit threshold
    maximize='Return [%]',         # Optimization objective: maximize return percentage
    max_tries=500,                 # Maximum number of optimization iterations
    random_state=42,               # Ensure reproducibility
    return_heatmap=True,           # Return a heatmap of all tried parameter combinations
    return_optimization=True,      # Return the full optimization result object
    method='skopt'                 # Use Bayesian optimization (via scikit-optimize)
)

/var/folders/1g/42ndnt6971569259087l_v9w0000gn/T/ipykernel_52202/416841959.py:2: DeprecationWarning: `Backtest.optimize(method="skopt")` is deprecated. Use `method="sambo"`.
  stats_skopt, heatmap, optimize_result = bt.optimize(


In [80]:
# Convert the heatmap result into a DataFrame for inspection
dff = heatmap.reset_index()
# Sort by return (descending) to identify the best parameter combinations
dff = dff.sort_values('Return [%]', ascending=False)

dff.head(n=5)

,limit_buy,limit_sell,Return [%]
15,3,-6,158.643092
9,2,-6,93.087804
3,1,-6,88.013628
0,0,-6,70.746751
16,3,-5,50.885759


---
#### V. Unanchored Walk Forward

Create a library of strategies. Create the unanchored walk forward class.

Everything can be checked on `iOStrategies.py`. Now import the strategy and perform the backtest.

In [81]:
%load_ext autoreload
%autoreload 2
import iOStrategies
iOStrategies.WalkForwardUnanchored

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


iOStrategies.WalkForwardUnanchored

In [82]:
bt_unanchored = Backtest(df, iOStrategies.WalkForwardUnanchored, cash=10000, commission=.002, exclusive_orders=True)

stats_skopt, heatmap, optimize_result = bt_unanchored.optimize(
    limit_buy = range(0, 6), limit_sell = range(-6, 0),
    maximize='Return [%]',
    max_tries=500,
    random_state=42,
    return_heatmap=True,
    return_optimization=True,
    method='skopt'
    )

/var/folders/1g/42ndnt6971569259087l_v9w0000gn/T/ipykernel_52202/1560995266.py:3: DeprecationWarning: `Backtest.optimize(method="skopt")` is deprecated. Use `method="sambo"`.
  stats_skopt, heatmap, optimize_result = bt_unanchored.optimize(


In [83]:
dff = heatmap.reset_index()
dff = dff.sort_values('Return [%]', ascending=False)
dff.head(n=5)

,limit_buy,limit_sell,Return [%]
9,2,-6,93.087804
0,0,-6,83.275499
3,1,-6,76.073826
15,3,-6,66.142228
23,4,-4,24.176106


---
#### VI. Interpret the strategies' performance

In [84]:
bt.plot(filename='backtests/walk_forward_anchored.html')

GridPlot(id='p3097', ...)

<p align="center">
  <img src="screen/backtest_report_MSFT_AWF.png" width="800"/>
</p>

In [85]:
bt_unanchored.plot(filename='backtests/walk_forward_unanchored.html')

GridPlot(id='p3451', ...)

<p align="center">
  <img src="screen/backtest_report_MSFT_UWF.png" width="800"/>
</p>